In [1]:
import numpy as np
import pandas as pd 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
os.path.join(PROJECT_ROOT_DIR)
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
CLEAREDDATA_PATH = os.path.join(PROJECT_ROOT_DIR, "cleareddata")
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(CLEAREDDATA_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from pipe_classes import *

./data/scotch_review.csv
./data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data = pd.read_excel(PROJECT_ROOT_DIR+'/data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx','Information and Metrics')

In [3]:
data = data.replace({'Not Available': np.nan})

In [4]:
data.select_dtypes(exclude=['number']).keys()

Index(['Property Name', 'Parent Property Id', 'Parent Property Name',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Address 2 (self-reported)', 'Postal Code', 'Street Number',
       'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type', '2nd Largest Property Use Type',
       '3rd Largest Property Use Type', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'Annual Maximum Demand (MM/YYYY)',
       'Water Required?', 'Generation Date',
       'DOF Benchmarking Submission Status'],
      dtype='object')

### First exercise: Lasso and ElasticNet without PCA or collinear features remover and without the sqrt-log feature adder

In [5]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

reg_target="ENERGY STAR Score"
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 95)

In [7]:
def results_display(reg, X, X_test, y, y_test):
    from sklearn.metrics import mean_squared_error, r2_score
    from timeit import default_timer as timer
    start = timer()
    reg.fit(X, y)
    #train set
    X_pred = reg.predict(X)
    X_rmse = np.sqrt(mean_squared_error(y, X_pred))
    X_R2 = r2_score(y, X_pred)
    #test set
    X_test_pred = reg.predict(X_test)
    X_test_rmse = np.sqrt(mean_squared_error(y_test, X_test_pred))
    X_test_R2 = r2_score(y_test, X_test_pred)
    end = timer()
    print(str(reg))
    print('Train set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_rmse,3), round(X_R2,3))
    print('Test set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_test_rmse,3), round(X_test_R2,3))
    print('Time consumption [s]: ', round((end - start),3))
    print('---------------------------------------------------------')

In [8]:
from sklearn.linear_model import Lasso
base_reg=Lasso(alpha=1.0, fit_intercept=True, normalize=False, precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

from sklearn.linear_model import ElasticNet
base_reg2=ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')

from sklearn.model_selection import GridSearchCV

In [9]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4335.252245371696, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12842.812687030062, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9591.929673506413, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 883722.00965240

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3644.8074388918467, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10117.76447986206, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3470.494804845657, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9544.87297401

Best estimator: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=500,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'max_iter': 500}
---------------------------------------------------------------------
18.048306671940594 {'alpha': 1.0, 'max_iter': 200}
18.048021913851716 {'alpha': 1.0, 'max_iter': 500}
18.048021913851716 {'alpha': 1.0, 'max_iter': 1000}
18.048021913851716 {'alpha': 1.0, 'max_iter': 2000}
17.395068921873893 {'alpha': 0.1, 'max_iter': 200}
17.39467804790433 {'alpha': 0.1, 'max_iter': 500}
17.39467804790433 {'alpha': 0.1, 'max_iter': 1000}
17.39467804790433 {'alpha': 0.1, 'max_iter': 2000}
17.34763692241368 {'alpha': 0.01, 'max_iter': 200}
17.344769985240273 {'alpha': 0.01, 'max_iter': 500}
17.34383517386449 {'alpha': 0.01, 'max_iter': 1000}
17.34376158381778 {'alpha': 0.01, 'max_iter': 2000}
17.3480465376241 {'alpha': 0.001, 'max_iter': 200}
17

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1976151.1625783397, tolerance: 1770.531869669478
  positive)


Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=500,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Train set: Regressor name, RMSE, R2
Lasso 17.276 0.708
Test set: Regressor name, RMSE, R2
Lasso 17.764 0.689
Time consumption [s]:  0.649
---------------------------------------------------------
---------------------------------------------------------------------


...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1749494.8422003696, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 408823.4299383438, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 671123.0977854638, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 455742.2065163

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2058360.5448086325, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2059175.5356780682, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2071970.994113923, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2060930.9884

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1489482.5321509917, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1918894.9246283195, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1474141.072076357, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1849864.967613

Best estimator: ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'l1_ratio': 0.7, 'max_iter': 1000}
---------------------------------------------------------------------
17.369316494345288 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
17.370618900411234 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
17.36173730710027 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
17.363068240108205 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
17.355137111767476 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 20

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 395482.7584486874, tolerance: 1770.531869669478
  positive)


first observations:
- we have 95 features
- both Lasso and ElasticNet perform quite good and are fast
- both Lasso and ElasticNet show R2-scores of ~71% on training set and ~69% on test set

### Second exercise: Lasso and ElasticNet with the sqrt-log feature adder

In [10]:
spec_cols=['Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 143)
(24724, 143)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 142)

In [12]:
X_test = num_pipeline.fit_transform(X_test)

In [13]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5629.113326133229, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2354.624914081767, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2370.471618469339, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3983.959935400

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1641839.248712887, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1663842.742752234, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1564957.6736185714, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1581490.8946964

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1777454.5092600458, tolerance: 1770.531869669478
  positive)


Best estimator: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=2000,
      normalize=False, positive=False, precompute=False, random_state=4711,
      selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'max_iter': 2000}
---------------------------------------------------------------------
17.174082230798987 {'alpha': 1.0, 'max_iter': 200}
17.174082230798987 {'alpha': 1.0, 'max_iter': 500}
17.174082230798987 {'alpha': 1.0, 'max_iter': 1000}
17.174082230798987 {'alpha': 1.0, 'max_iter': 2000}
16.3449737989269 {'alpha': 0.1, 'max_iter': 200}
16.344354728069458 {'alpha': 0.1, 'max_iter': 500}
16.344354728069458 {'alpha': 0.1, 'max_iter': 1000}
16.344354728069458 {'alpha': 0.1, 'max_iter': 2000}
16.186167133341154 {'alpha': 0.01, 'max_iter': 200}
16.162546952918923 {'alpha': 0.01, 'max_iter': 500}
16.149887610036345 {'alpha': 0.01, 'max_iter': 1000}
16.142958763776946 {'alpha': 0.01, 'max_iter': 2000}
16.211427471780638 {'alpha': 0.001, 'max_iter':

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1777454.5092600458, tolerance: 1770.531869669478
  positive)


...

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1604138.8644401655, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1582296.6349639783, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1586737.1435903814, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1635655.883

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1797224.3341096293, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1805445.2179992395, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1762480.866739488, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1768661.308176

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1733129.7794682242, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1738606.225281436, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1728705.0758278384, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1751903.935

Best estimator: ElasticNet(alpha=0.0001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=2000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.0001, 'l1_ratio': 0.7, 'max_iter': 2000}
---------------------------------------------------------------------
16.24344555475759 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
16.255854980329683 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
16.254209353835908 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
16.25462429459646 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
16.223528176189422 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
16.231150029224548 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
16.23237258498199 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
16.23222640845116 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
16.204200971042653 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 200

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2182030.0941083287, tolerance: 1770.531869669478
  positive)


observations:
- we have 142 (base: 95) features
- the regression is factor 2 slower as the beseline regression
- both Lasso and ElasticNet show performance improvement with log and sqrt features added
- both Lasso and ElasticNet show R2-scores of ~75% (base: 71%) on training set and ~73% (base: 69%) on test set

### Third exercise: Lasso and ElasticNet with the sqrt-log feature adder and the collinear features remover

In [14]:
spec_cols=['Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('collinearfeatures_remover', CollinearFeatures_Remover(reg_target)),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 83)
(24724, 83)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 82)

In [16]:
X_test = num_pipeline.fit_transform(X_test)

In [17]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 58280.95594453113, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 49411.57093198644, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51479.01236866694, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46876.1237073

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 49771.511173128616, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 45208.03078032285, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 64815.10323592229, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 64122.625145833

Best estimator: Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=200, normalize=False,
      positive=False, precompute=False, random_state=4711, selection='cyclic',
      tol=0.0001, warm_start=False)
Best params: {'alpha': 0.1, 'max_iter': 200}
---------------------------------------------------------------------
19.339957588723955 {'alpha': 1.0, 'max_iter': 200}
19.339957588723955 {'alpha': 1.0, 'max_iter': 500}
19.339957588723955 {'alpha': 1.0, 'max_iter': 1000}
19.339957588723955 {'alpha': 1.0, 'max_iter': 2000}
18.747685725264606 {'alpha': 0.1, 'max_iter': 200}
18.747685725264606 {'alpha': 0.1, 'max_iter': 500}
18.747685725264606 {'alpha': 0.1, 'max_iter': 1000}
18.747685725264606 {'alpha': 0.1, 'max_iter': 2000}
18.748287744421592 {'alpha': 0.01, 'max_iter': 200}
18.748424386920227 {'alpha': 0.01, 'max_iter': 500}
18.748424386920227 {'alpha': 0.01, 'max_iter': 1000}
18.748424386920227 {'alpha': 0.01, 'max_iter': 2000}
18.749659504875257 {'alpha': 0.001, 'max_iter': 200

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 33928.32367894985, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 35187.79670612328, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 36302.39176215418, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 69006.1319102938

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 236745.1385193509, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 258401.21462616324, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 279453.5324483402, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 237231.99153

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1271759.3804484562, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1313871.6429240892, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1364772.716232445, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1285373.302

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 400658.69207877386, tolerance: 1424.8921661249544
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 348685.3242639811, tolerance: 1414.366648941856
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 386052.2717154119, tolerance: 1413.68276143012
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 355764.50307445

Best estimator: ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.7,
           max_iter=2000, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.001, 'l1_ratio': 0.7, 'max_iter': 2000}
---------------------------------------------------------------------
18.757188213346634 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
18.757192643117804 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
18.757192643117804 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
18.757192643117804 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
18.752735258767096 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
18.752735258767096 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
18.752735258767096 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
18.752735258767096 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
18.74958497511735 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 20

observations:
- we have 82 (base: 95) features
- the regression is nearly as fast as the beseline regression
- both Lasso and ElasticNet show R2-scores of ~66% (base: 71%; loqsqrt: 75%) on training set and ~65% (base: 69%; loqsqrt: 73%) on test set

### Forth exercise: Lasso and ElasticNet with the sqrt-log feature adder and the a PCA instead of the collinear features removal

In [18]:
spec_cols=['Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_adderencoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 143)
(24724, 143)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 142)

In [20]:
X_test = num_pipeline.fit_transform(X_test)

In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_reduced = pca.fit(X)
pca.explained_variance_ratio_

array([0.21276408, 0.10829207, 0.07678783, 0.04403533, 0.03170018,
       0.02989606, 0.02841492, 0.0277648 , 0.02484918, 0.02350585,
       0.02086465, 0.01854084, 0.01587575, 0.01287099, 0.01226055,
       0.01150367, 0.01091358, 0.01060738, 0.01046451, 0.01038366,
       0.01028216, 0.01026388, 0.01025472, 0.0102418 , 0.0102219 ,
       0.01021581, 0.01020954, 0.01020814, 0.01020498, 0.01020397,
       0.01019574, 0.0101727 , 0.01013158, 0.01011424, 0.01003206,
       0.0099693 , 0.00975885, 0.00959698, 0.00889919, 0.00761854,
       0.00753078, 0.00677738])

In [22]:
1 - pca.explained_variance_ratio_.sum()

0.044599877963056755

In [23]:
X_reduced = pca.transform(X)
X_test_reduced = pca.transform(X_test)
X_reduced.shape

(17306, 42)

In [24]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_reduced, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X_reduced, X_test_reduced, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_reduced, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
print('---------------------------------------------------------------------')
results_display(grid_best_est, X_reduced, X_test_reduced, y, y_test)
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1626468.3602643549, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 406113.499537752, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 584149.7255887194, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 366741.677627

Best estimator: Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=200, normalize=False,
      positive=False, precompute=False, random_state=4711, selection='cyclic',
      tol=0.0001, warm_start=False)
Best params: {'alpha': 0.1, 'max_iter': 200}
---------------------------------------------------------------------
18.94270743988004 {'alpha': 1.0, 'max_iter': 200}
18.94270743988004 {'alpha': 1.0, 'max_iter': 500}
18.94270743988004 {'alpha': 1.0, 'max_iter': 1000}
18.94270743988004 {'alpha': 1.0, 'max_iter': 2000}
18.686200620476676 {'alpha': 0.1, 'max_iter': 200}
18.686200620476676 {'alpha': 0.1, 'max_iter': 500}
18.686200620476676 {'alpha': 0.1, 'max_iter': 1000}
18.686200620476676 {'alpha': 0.1, 'max_iter': 2000}
69.43985412573993 {'alpha': 0.01, 'max_iter': 200}
97.22798711752141 {'alpha': 0.01, 'max_iter': 500}
109.70963787696688 {'alpha': 0.01, 'max_iter': 1000}
126.64822430795766 {'alpha': 0.01, 'max_iter': 2000}
85.93439262354845 {'alpha': 0.001, 'max_iter': 200}
136.9

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 558659.9482614882, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 774007.2264700313, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1092315.8839120755, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6103.0552698

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2305930.995129788, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2050831.1247530514, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2286815.173237145, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2149755.1043

Best estimator: ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.3,
           max_iter=200, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
---------------------------------------------------------------------
33.47593627752629 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
33.80351313734318 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
33.80351313734318 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
33.80351313734318 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
38.51446250237922 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
39.57045798557497 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
39.57045798557497 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
39.57045798557497 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
46.321894372877 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 200}
49.76543051

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2259883.9719939143, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


observations:
- we have 42 (base: 95) features
- the regression is the fastest up to now
- both Lasso and ElasticNet show R2-scores of ~67% (base: 71%; loqsqrt: 75%; logsqrtcolfeatrem: 66%) on training set and ~65% (base: 69%; loqsqrt: 73%; logsqrtcolfeatrem: 65%) on test set; so it is even a bit better!

### Fifth exercise: Lasso and ElasticNet just with PCA

In [25]:
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 95)

In [27]:
X_test = num_pipeline.fit_transform(X_test)

In [28]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_reduced = pca.fit(X)
pca.explained_variance_ratio_

array([0.13485975, 0.07365993, 0.0679767 , 0.04655914, 0.03601356,
       0.02643084, 0.02513202, 0.02303103, 0.02197663, 0.02126157,
       0.02053306, 0.02013216, 0.02000591, 0.01981269, 0.01974769,
       0.01969423, 0.01966451, 0.01965091, 0.01962738, 0.01962406,
       0.01962021, 0.01961771, 0.01961391, 0.01961024, 0.01960902,
       0.01960086, 0.01957536, 0.01951966, 0.01943736, 0.01912276,
       0.01881726, 0.01867528, 0.0182599 , 0.01741083, 0.01586174])

In [29]:
1 - pca.explained_variance_ratio_.sum()

0.04025410605096591

In [30]:
X_reduced = pca.transform(X)
X_test_reduced = pca.transform(X_test)
X_reduced.shape

(17306, 35)

In [31]:
param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

lasso_reg = Lasso(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_reduced, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X_reduced, X_test_reduced, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 32 (3×4x3) combinations of hyperparameters
    {'alpha': [0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000], 'l1_ratio': [0.3, 0.5, 0.7]},
    ]

elnet_reg = ElasticNet(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 32*5=160 rounds of training 
grid_search = GridSearchCV(elnet_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_reduced, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
print('---------------------------------------------------------------------')
results_display(grid_best_est, X_reduced, X_test_reduced, y, y_test)
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1818830.3443229166, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 882014.4965735306, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1623783.6924090819, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 729678.6718

Best estimator: Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=200, normalize=False,
      positive=False, precompute=False, random_state=4711, selection='cyclic',
      tol=0.0001, warm_start=False)
Best params: {'alpha': 0.1, 'max_iter': 200}
---------------------------------------------------------------------
19.815353227851293 {'alpha': 1.0, 'max_iter': 200}
19.815353227851293 {'alpha': 1.0, 'max_iter': 500}
19.815353227851293 {'alpha': 1.0, 'max_iter': 1000}
19.815353227851293 {'alpha': 1.0, 'max_iter': 2000}
19.41223118674027 {'alpha': 0.1, 'max_iter': 200}
19.41223118674027 {'alpha': 0.1, 'max_iter': 500}
19.41223118674027 {'alpha': 0.1, 'max_iter': 1000}
19.41223118674027 {'alpha': 0.1, 'max_iter': 2000}
63.717915736691424 {'alpha': 0.01, 'max_iter': 200}
128.88375646958593 {'alpha': 0.01, 'max_iter': 500}
205.64696939106878 {'alpha': 0.01, 'max_iter': 1000}
288.3322004864265 {'alpha': 0.01, 'max_iter': 2000}
110.3001361058787 {'alpha': 0.001, 'max_iter': 200}
241.

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 226285.64228341728, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 439901.48567499034, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 922028.1466605826, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10704.48847

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2417810.6530617625, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2534542.19363364, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2362846.120887264, tolerance: 1419.0815962077436
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2510483.41187

Best estimator: ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.3,
           max_iter=200, normalize=False, positive=False, precompute=False,
           random_state=4711, selection='cyclic', tol=0.0001, warm_start=False)
Best params: {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
---------------------------------------------------------------------
23.911723632578347 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 200}
24.01393016051805 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 500}
24.01393016051805 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 1000}
24.01393016051805 {'alpha': 0.01, 'l1_ratio': 0.3, 'max_iter': 2000}
26.214974123154224 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 200}
26.68182956100267 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 500}
26.68182956100267 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 1000}
26.68182956100267 {'alpha': 0.01, 'l1_ratio': 0.5, 'max_iter': 2000}
30.91293830356747 {'alpha': 0.01, 'l1_ratio': 0.7, 'max_iter': 200}
33.4668

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2431571.3003795063, tolerance: 1410.0382446081626
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


observations:
- we have 35 (base: 95) features
- the regression is by farthe fastest
- both Lasso and ElasticNet show R2-scores of ~63% (base: 71%; loqsqrt: 75%; logsqrtcolfeatrem: 66%; logsqrtpca: 67%) on training set and ~62% (base: 69%; loqsqrt: 73%; logsqrtcolfeatrem: 65%; logsqrtpca: 65%) on test set